In [2]:
from datasets import Dataset
import pandas as pd
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import LoraConfig, TaskType, get_peft_model
from swanlab.integration.huggingface import SwanLabCallback
import swanlab
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-07 17:41:07.897768: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 17:41:07.909807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 17:41:07.924154: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 17:41:07.928491: E external/l

In [3]:
# 将JSON文件转换为CSV文件
# 将JSON文件转换为CSV文件
# df = pd.read_json('./news_train.jsonl',lines = True)
df = pd.read_csv('./data/xhsid20_shangpin_pinpai_1000.csv')
# df['instruction'] = """
#             任务说明：
#             你是一个文章品牌提取助手，你的任务是通过分析我提供的文章数据，提取文章中主要讲解的商品及商品品牌。
#             请仔细阅读文章内容，并识别出文中提到的所有主要品牌名称及商品。请列出这些商品和品牌，该过程将分为两个主要步骤：
#             1.判断是否为商品文章：
#                 判断依据：文章是否直接介绍或推销某个商品。
#                 若不满足该标准，则认为不是商品文章。
#             2.对商品文章数据提取商品和品牌：
#                 确保提取的品牌是文章中主要介绍的商品和品牌，输出形式为商品-品牌。

#             提取规则
#             文章数据识别: 主要通过对文章的主体对象判断是否直接介绍或推销某个商品而不是介绍非商品内容或者偶像组合。
#             品牌识别： 需要准确判断文章中是否含有商品品牌。
#             品牌完整性： 文章可能涉及多个商品品牌，确保输出品牌的完整性，用户名不作为商品品牌。
#             门店名称提取： 如果文章为个体户门店，请确保能提取出门店名称。
#             正确分类：主要分为三类，不是商品文章，有商品但无品牌，有品牌但无商品，有商品和品牌。
#             根据品牌沿伸商品：对于文章中只有品牌没有商品的，则需要通过文章内容，获取最可能的商品

#             输出标签格式如：
#             若包含品牌则输出所有可能的品牌:["商品1-品牌名1", "商品2-品牌名2", ...]。
#             若商品文章数据不包含品牌则输出:["无商品品牌"]。
#             若不是商品文章数据则输出:["不是商品文章"]。
#             """
df['instruction'] = """
                         任务说明：
             你是一个文章商品和品牌提取助手，你的任务是通过分析我提供的文章数据，提取文章中的商品及对应品牌。
             请仔细阅读文章内容，并识别出文中提到的所有商品和品牌。请列出这些商品和品牌，该过程将分为两个主要步骤：
             1.判断是否为商品文章：
                 判断依据：文章是否直接介绍或推销某个商品。
                 若不满足该标准，则认为不是商品文章，输出格式为["不是商品文章"]。
             2.对商品文章数据提取商品和品牌：
                 分情况：若商品和品牌都存在，则输出格式为["商品名-品牌名", "商品名-品牌名", ...]
                        若只存在商品，则输出格式为["无商品品牌"]
                        若只存在品牌，则根据文章内容生成对应品牌的商品，输出格式为["生成的商品名-品牌名", "生成的商品名-品牌名", ..., "生成的商品名-品牌名"]

             提取规则
             文章数据识别: 主要通过对文章的主体对象判断是否直接介绍或推销某个商品而不是介绍非商品内容或者偶像组合。
             品牌识别： 需要准确判断文章中是否含有商品品牌。
             品牌完整性： 文章可能涉及多个商品品牌，确保输出品牌的完整性，用户名不作为商品品牌。
             门店名称提取： 如果文章为个体户门店，请确保能提取出门店名称。
             """
df = df[['instruction','input', 'output']]
ds = Dataset.from_pandas(df)
test_df = df[50:110]  # 取前10条做主观评测

In [4]:
test_df.to_json('./autodl-tmp/data/shangpinpinpai_test.json', orient='records',indent=4)

In [3]:
test_df

,instruction,input,output
90,\n 任务说明：\n ...,"以前为了怕卡粉厚重 底妆一直追求轻薄 结果遮瑕又弱妆效也一般,这次用zati粉底液就多涂了一...","[“粉底液-zati""]"
91,\n 任务说明：\n ...,# 吗喽 # 动物 # 可爱,[“不是商品文章”]
92,\n 任务说明：\n ...,威飒的黑色内饰提供经典、时尚的外观，并且耐脏，无需勤打理。座舱采用T字型对称式布局，视觉上宽...,"[“内饰-威飒""]"
93,\n 任务说明：\n ...,1、aisei珍珠米白，很通透的大直径玻璃珠，日常戴不容易出错\n2、aisei月见茶灰，清...,"[“美瞳-aisei""]"
94,\n 任务说明：\n ...,😱谁家电视这么敢的啊！？\n🔥海信E8N Ultra限时免费安装📍新品首发强势来袭！\n⭕...,"[“电视-海信""]"
95,\n 任务说明：\n ...,摩拉尤品这个新品水牛奶[哇R]好喝得像液体奶酪!!\n[自拍R]烫金礼盒装，春天的配色，颜值...,"[“水牛奶-摩拉尤品""]"
96,\n 任务说明：\n ...,海口现货❤️100ml到手3360❤️50ml到手1860❤️日期新鲜哟❤️送人🈶礼袋，礼盒...,[“不是商品文章”]
97,\n 任务说明：\n ...,💖•新店入驻💗\n🥳霸王茶姬 化州首店🤗\n[咖啡]打造东⽅新茶铺💕\n⏰即将开业 敬请期待🎉,"[“奶茶-霸王茶姬""]"
98,\n 任务说明：\n ...,摩圈、MOTO图库、公众号、抖音搜索:谷GU追焦 [doge],[“不是商品文章”]
99,\n 任务说明：\n ...,原价出两张5.18惠州dearjane\n本来5.16要去福州看周杰伦 然后5.17要去厦门...,[“不是商品文章”]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  980 non-null    object
 1   input        980 non-null    object
 2   output       980 non-null    object
dtypes: object(3)
memory usage: 23.1+ KB


In [8]:
df[df['output'].isnull()]

,instruction,input,output


# 处理数据集

In [9]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Tokenizer(name_or_path='/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [11]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 980/980 [00:03<00:00, 245.60 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 980
})

In [12]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n\n                         任务说明：\n             你是一个文章商品和品牌提取助手，你的任务是通过分析我提供的文章数据，提取文章中的商品及对应品牌。\n             请仔细阅读文章内容，并识别出文中提到的所有商品和品牌。请列出这些商品和品牌，该过程将分为两个主要步骤：\n             1.判断是否为商品文章：\n                 判断依据：文章是否直接介绍或推销某个商品。\n                 若不满足该标准，则认为不是商品文章，输出格式为["不是商品文章"]。\n             2.对商品文章数据提取商品和品牌：\n                 分情况：若商品和品牌都存在，则输出格式为["商品名-品牌名", "商品名-品牌名", ...]\n                        若只存在商品，则输出格式为["无商品品牌"]\n                        若只存在品牌，则根据文章内容生成对应品牌的商品，输出格式为["生成的商品名-品牌名", "生成的商品名-品牌名", ..., "生成的商品名-品牌名"]\n\n             提取规则\n             文章数据识别: 主要通过对文章的主体对象判断是否直接介绍或推销某个商品而不是介绍非商品内容或者偶像组合。\n             品牌识别： 需要准确判断文章中是否含有商品品牌。\n             品牌完整性： 文章可能涉及多个商品品牌，确保输出品牌的完整性，用户名不作为商品品牌。\n             门店名称提取： 如果文章为个体户门店，请确保能提取出门店名称。\n             <|im_end|>\n<|im_start|>user\n上次护肤Roution分享了褒曼2号，还剩三分之一，续了Acaderma的平衡稳健水，看图片用量我还在磨合哈哈，现在换季购入了purid的舒润露，这篇详细说下感受：\n一\n洁面完用护肤水时通常'

In [13]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

''

In [14]:
type(tokenized_id[0]['input_ids'][0])

int

# 创建模型

In [15]:
device = "cuda"
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2-7B-Instruct',device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████| 4/4 [01:24<00:00, 21.16s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [16]:
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

In [17]:
model.dtype

torch.bfloat16

# Lora

In [18]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 传统语言模型
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1  # Dropout 比例
)

In [19]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'up_proj', 'down_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [20]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'up_proj', 'down_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [21]:
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


# 配置训练参数

In [22]:
args = TrainingArguments(
    output_dir="./Qwen/Qwen2_pinpai_1000",
    logging_dir='./Qwen/logs',
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    save_steps=10,  # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [23]:
swanlab_callback = SwanLabCallback(
    project="Qwen2-fintune",
    experiment_name="Qwen2-7B-Instruct-pinpai_1000",
    description="使用通义千问Qwen2-7B-Instruct模型在pinpai1000上prompt微调。",
    config={
        "model": "Qwen/Qwen2-7B-Instruct",
        "dataset": "xhspinpai.json",
    }
)

In [24]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback]
)
trainer.train()



Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2024-08-01 15:01:22,909] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  please install triton==1.0.0 if you want to use sparse attention
swanlab: Tracking run with swanlab version 0.3.9                                   
swanlab: Run data will be saved locally in /mnt/workspace/swanlog/run-20240801_150126-a3b1799d
swanlab: 👋 Hi zs-train, welcome to swanlab!
swanlab: Syncing run Qwen2-7B-Instruct-pinpai_1000_Aug01_15-01-26 to the cloud
swanlab: 🌟 Run `swanlab watch -l /mnt/workspace/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@zs-train/Qwen2-fintune
swanlab: 🚀 View run at https://swanlab.cn/@zs-train/Qwen2-fintune/runs/1ak1fest0vzqqzvr2jab7



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,3.627200
2,5.033000
3,4.342800
4,2.002400
5,2.020200
6,1.839900
7,0.741900
8,0.992600
9,1.027800
10,2.027500


/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/u

swanlab: Step 305 on key train/epoch already exists, ignored.


TrainOutput(global_step=305, training_loss=0.24218164794690922, metrics={'train_runtime': 1859.3728, 'train_samples_per_second': 2.635, 'train_steps_per_second': 0.164, 'total_flos': 7.969547737228493e+16, 'train_loss': 0.24218164794690922, 'epoch': 4.979591836734694})

swanlab: 🌟 Run `swanlab watch -l /mnt/workspace/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@zs-train/Qwen2-fintune
swanlab: 🚀 View run at https://swanlab.cn/@zs-train/Qwen2-fintune/runs/1ak1fest0vzqqzvr2jab7


In [25]:
swanlab.finish()

# 训练完推理

In [4]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = 'Qwen/Qwen2-7B-Instruct'
lora_path = './Qwen/Qwen2_pinpai_1000/checkpoint-300' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


## 加载完预测

In [5]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)

    return response


test_text_list = []
for index, row in test_df.iterrows():
    instruction = row["instruction"]
    input_value = row["input"]

    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"},
    ]

    response = predict(messages, model, tokenizer)
    messages.append({"role": "assistant", "content": f"{response}"})
    result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
    test_text_list.append(swanlab.Text(result_text, caption=response))
swanlab.init(
    project="Qwen2-fintune",
    experiment_name="Qwen2-7B-Instruct-pinpai_1000",
    description="使用通义千问Qwen2-7B-Instruct模型在pinpai1000微调。",
    config={
        "model": "Qwen/Qwen2-7B-Instruct",
        "dataset": "xhspinpai.json",
    })
swanlab.log({"Prediction": test_text_list})
swanlab.finish()

[“粉底液-zati"]
[“不是商品文章”]
[“内饰-威飒"]
[“美瞳-aisei”]
[“电视-海信"]
[“水牛奶-摩拉尤品"]
[“不是商品文章”]
[“奶茶-霸王茶姬"]
[“不是商品文章”]
[“票-dearjane”]
[“饮料-娃哈哈"]
[“衣服-stussy"]
[“二手车-福特"]
[“套装-安妮家"]
[“不是商品文章”]
[“珠宝-六福"]
[“不是商品文章”]
[“不是商品文章”]
[“手机-苹果"]
[“相机-富士"]
swanlab: Tracking run with swanlab version 0.3.9                                   
swanlab: Run data will be saved locally in /mnt/workspace/swanlog/run-20240801_153536-9fabef00
swanlab: 👋 Hi zs-train, welcome to swanlab!
swanlab: Syncing run Qwen2-7B-Instruct-pinpai_1000_Aug01_15-35-36 to the cloud
swanlab: 🌟 Run `swanlab watch -l /mnt/workspace/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@zs-train/Qwen2-fintune
swanlab: 🚀 View run at https://swanlab.cn/@zs-train/Qwen2-fintune/runs/7c0e6h5409pqwrisvv02e


swanlab: 🌟 Run `swanlab watch -l /mnt/workspace/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@zs-train/Qwen2-fintune
swanlab: 🚀 View run at https://swanlab.cn/@zs-train/Qwen2-fintune/runs/7c0e6h5409pqwrisvv02e



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
prompt = '''你叫什么名字'''
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个文本分类领域的专家，你会接收到一段文本和几个潜在的分类选项，请输出文本内容的正确类型"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to(device)


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

人物相关


## webui

In [4]:
import gradio as gr

In [14]:
def continue_dialogue(user_input, reset=False):
    expert_prompt = "你是一个文本分类领域的专家，你会接收到一段文本并输出文本内容的正确类型，以下为给出的内容："
    inputs = tokenizer.apply_chat_template([{"role": "user", "content":expert_prompt+"\n"  + user_input}],
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    # 生成响应
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 2}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

title = "Transformer Chatbot"
description = "Enter your message and receive a response from the transformer-based language model."

iface = gr.ChatInterface(
    fn=continue_dialogue,
    title=title,
    description=description,
    examples=[["这里可以使用你微调的数据代替"],
              ["你叫什么名字?"]]
)
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/gradio/queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/site-packages/gradio/blocks.py", line 1512, in call_function
    prediction = await fn(*processed_input)
  File "/usr/local/lib/python3.10/site-packages/gradio/utils.py", line 800, in async_wrapper
    response = await f(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/gradio/chat_interface.py", line 554, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async

In [84]:
dialogue_history = []
def continue_dialogue(user_input, reset=False):
    global dialogue_history
    if reset:
        dialogue_history = []  # 重置对话历史
    dialogue_history.append({"role": "user", "content": "你是一个文本分类领域的专家，你会接收到一段文本并输出文本内容的正确类型，以下为给出的内容："})
    dialogue_history.append({"role": "user", "content": user_input})
    # 生成当前对话的输入
    inputs = tokenizer.apply_chat_template(dialogue_history,
                                           add_generation_prompt=True,
                                           tokenize=True,
                                           return_tensors="pt",
                                           return_dict=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    # 生成响应
    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 3}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    dialogue_history.append({"role": "model", "content": response})
    return response


# def reset_dialogue():
    # continue_dialogue("", reset=True)
    # return "对话历史已重置！"


title = "Transformer Chatbot"
description = "Enter your message and receive a response from the transformer-based language model."
iface = gr.ChatInterface(
    fn=continue_dialogue,
    title=title,
    description=description,
    examples=[["这里可以使用你微调的数据代替"],
              ["你叫什么名字?"]]
)
iface.launch()

Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.
